In [9]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy.io import arff
import pandas as pd
from scipy import stats
import itertools
import random
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

#My imports
import h5py

In [10]:
# arr = ('/eos/cms/store/group/comm_dqm/cmsml4dc/2016-v1/data2016_v2/'+
#       'BTagCSV/crab_BTagCSVRun_2016C-18Apr2017-v1/170808_213346/0000/AODTree_102.npy')
good_arr = '/mnt/hdf5test/BTagCSV_C_signal.h5'
bad_arr  = '/mnt/hdf5test/BTagCSV_C_background.h5'

# data = np.load(arr, encoding = 'latin1')
sig_jetfile = h5py.File(good_arr,'r')
sig_jet     = sig_jetfile['BTagCSV_C_signal'[:]]

bg_jetfile = h5py.File(bad_arr,'r')
bg_jet     = bg_jetfile['BTagCSV_C_background'[:]]

good_jets = np.empty([0,2802])
bad_jets  = np.empty([0,2802])
# print(bad_jets)
print("XXXXXX")
good_jets = np.concatenate((good_jets, sig_jet), axis=0)
bad_jets  = np.concatenate((bad_jets,  bg_jet),  axis=0)
# print(bad_jets)

df1 = pd.DataFrame(good_jets)
print(df1)
df1['class'] = 1
print(df1)
df2 = pd.DataFrame(bad_jets)
df2['class'] = 0

frames = [df1,df2]
df     = pd.concat(frames)
print(df)
# # arr = np.random.rand(10, 5)
# df = pd.DataFrame(data)
# print(df)

XXXXXX
            0          1         2         3         4          5     \
0      14.869158  26.557335  5.000006  6.939612  9.511660  13.633229   
1      14.926449  27.545649  5.000018  6.915446  9.435931  13.510402   
2      14.842835  27.755945  5.000005  6.762529  9.107812  12.951828   
3      14.722753  27.691078  5.000108  6.763557  9.096086  12.962060   
4      14.875105  28.402660  5.000045  6.770486  9.103080  12.963690   
5      14.911256  28.526539  5.000014  6.755152  9.090521  12.945515   
6      14.982747  25.633022  5.000001  7.109210  9.867184  14.214623   
7      15.060741  26.060837  5.000038  7.089802  9.839666  14.167170   
8      15.128889  27.928976  5.000000  7.010808  9.644757  13.852316   
9      14.969275  26.351051  5.000033  7.016041  9.661500  13.907710   
10     15.486270  32.177094  5.000020  6.277380  8.046266  11.352130   
11     15.631203  32.805599  5.000012  6.264451  8.045138  11.434741   
12     15.702190  32.797314  5.000236  6.247756  8.027581

In [11]:
cutted_df = df.iloc[:, 2600:2804]
df = cutted_df
print(df)

     2600      2601      2602         2603      2604       2605        2606  \
0     0.0  0.061303  0.207760   654.261780  0.028592  18.044600 -166.469971   
1     0.0  0.060519  0.206296  1966.122314  0.028759  18.116125 -127.100533   
2     0.0  0.060096  0.205639   377.614990  0.051212  17.778246 -159.067810   
3     0.0  0.059871  0.205306   596.146240  0.018815  17.729181 -160.941010   
4     0.0  0.059911  0.205938   388.404816  0.025332  17.802599 -134.671951   
5     0.0  0.060825  0.207431  1190.637329  0.034126  17.749704 -124.790199   
6     0.0  0.062742  0.208965   379.036530  0.105394  18.408548 -165.152634   
7     0.0  0.062452  0.209003   656.924255  0.118649  18.372927 -133.848816   
8     0.0  0.061785  0.208765   906.402039  0.052506  18.124384 -140.210541   
9     0.0  0.061419  0.208309  1693.866455  0.089383  18.154665 -131.161209   
10    0.0  0.054235  0.194934   254.922775 -0.151770  16.863489 -125.189629   
11    0.0  0.054778  0.196804   535.986023 -0.143115

In [4]:
# kokot = df.iloc[:, :5]
# print(kokot)
# # kokot['class'] = 0
# kokot['class'] = np.random.randint(0,2, kokot.shape[0])

# picus = kokot
# print(picus)
# # df = kokot
# # print(df)
# # store = pd.read_hdf(sig_jet, key='BTagCSV_C_signal')
# df = picus
# print(pd.to_numeric(picus['class'].values))

In [5]:
# ann_thyroid, meta = arff.loadarff('ann_thyroid.arff')
# ann = pd.DataFrame(ann_thyroid)
# print(ann)

In [ ]:
%%time
#We use arff from scipy.io to read the file
#Download the data from here http://ipd.kit.edu/~muellere/HiCS/

# data, meta = arff.loadarff('ann_thyroid.arff')
# df = pd.DataFrame(data)

#Below functions are used to calculate LOF
def knn(df,k):
    nbrs = NearestNeighbors(n_neighbors=3)
    nbrs.fit(df)
    distances, indices = nbrs.kneighbors(df)
    return distances, indices

def reachDist(df,MinPts,knnDist):
    nbrs = NearestNeighbors(n_neighbors=MinPts)
    nbrs.fit(df)
    distancesMinPts, indicesMinPts = nbrs.kneighbors(df)
    distancesMinPts[:,0] = np.amax(distancesMinPts,axis=1)
    distancesMinPts[:,1] = np.amax(distancesMinPts,axis=1)
    distancesMinPts[:,2] = np.amax(distancesMinPts,axis=1)
    return distancesMinPts, indicesMinPts

def ird(MinPts,knnDistMinPts):
    return (MinPts/np.sum(knnDistMinPts,axis=1))

def lof(Ird,MinPts,dsts):
    lof=[]
    for item in dsts:
        tempIrd = np.divide(Ird[item[1:]],Ird[item[0]])
        lof.append(tempIrd.sum()/MinPts)
    return lof

#calculate the index, we use this for selecting random sections in subspace
index_df = (df.rank()/df.rank().max()).iloc[:,:-1]
print("index_df calculated")
#Below function is used to create possible subspaces in each step
def comboGenerator(startPoint,space,n):
    combosFinal=[]
    for item in itertools.combinations(list(set(space)-set(startPoint)),(n-len(startPoint))):
        combosFinal.append(sorted(startPoint+list(item)))
    return combosFinal

listOfCombos = comboGenerator([],df.columns[:-1],2)
print("lenght of listofcombos is ", len(listOfCombos))
testedCombos=[]
selection=[]
while(len(listOfCombos)>0):
    if listOfCombos[0] not in testedCombos:
#         print("Calculating {0}".format(listOfCombos[0]))
        alpha1 = pow(0.2,(float(1)/float(len(listOfCombos[0]))))
        pvalue_Total =0
        pvalue_cnt = 0
        avg_pvalue=0
        for i in range(0,50):
            lband = random.random()
            uband = lband+alpha1
            v = random.randint(0,(len(listOfCombos[0])-1))
            rest = list(set(listOfCombos[0])-set([listOfCombos[0][v]]))
            k=stats.ks_2samp(df[listOfCombos[0][v]].values, df[((index_df[rest]<uband) & (index_df[rest]>lband)).all(axis=1)][listOfCombos[0][v]].values)
#             print("iter:{4},lband:{0},uband:{1},v:{2},pvalue:{3},length:{5},rest:{6}".format(lband,uband,v,k.pvalue,i,len(df[((index_df[rest]<uband) & (index_df[rest]>lband)).all(axis=1)][listOfCombos[0][v]]),rest))
            if not(np.isnan(k.pvalue)):
                pvalue_Total = pvalue_Total+k.pvalue
                pvalue_cnt = pvalue_cnt+1
        if pvalue_cnt>0:
            avg_pvalue = pvalue_Total/pvalue_cnt
#             print(avg_pvalue)
        if (1.0-avg_pvalue)>0.75:
            selection.append(listOfCombos[0])
            listOfCombos = listOfCombos + comboGenerator(listOfCombos[0],df.columns[:-1],(len(listOfCombos[0])+1))
        testedCombos.append(listOfCombos[0])
        listOfCombos.pop(0)
        listOfCombos = [list(t) for t in set(map(tuple,listOfCombos))]
    else:
        listOfCombos.pop(0)
#     print("current lenght of listofcombos is ", len(listOfCombos))


scoresList=[]
print("Lenght of selection is ", len(selection))
for item in selection:
    m=50
    knndist, knnindices = knn(df[item],3)
    reachdist, reachindices = reachDist(df[item],m,knndist)
    irdMatrix = ird(m,reachdist)
    lofScores = lof(irdMatrix,m,reachindices)
    scoresList.append(lofScores)


avgs = np.nanmean(np.ma.masked_invalid(np.array(scoresList)),axis=0)

scaled_avgs = MinMaxScaler().fit_transform(avgs.reshape(-1,1))

print("HCiS AUC Score")
print(pd.to_numeric(df['class'].values))
# print(scaled_avgs)
print(metrics.roc_auc_score(pd.to_numeric(df['class'].values),scaled_avgs))

m=50
knndist, knnindices = knn(df.iloc[:,:-1],3)
reachdist, reachindices = reachDist(df.iloc[:,:-1],m,knndist)
irdMatrix = ird(m,reachdist)
lofScores = lof(irdMatrix,m,reachindices)
ss=MinMaxScaler().fit_transform(np.array(lofScores).reshape(-1,1))
print("LOF AUC Score")
print(metrics.roc_auc_score(pd.to_numeric(df['class'].values),ss))

index_df calculated
lenght of listofcombos is  20301


/usr/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:4558: RuntimeWarning: invalid value encountered in true_divide
  cdf2 = np.searchsorted(data2, data_all, side='right') / (1.0*n2)
/usr/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:4563: RuntimeWarning: divide by zero encountered in double_scalars
  prob = distributions.kstwobign.sf((en + 0.12 + 0.11 / en) * d)
